In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

### Use cell mask and edt and sample background photons as as function of distance

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.generic.emitter import EmitterSet
from cellbgnet.train_loss_infer import generate_probmap_cells
from skimage.io import imread
from skimage.measure import label
import random
import edt
from skimage.filters import gaussian
import random

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.training(128, simulation_params)

sampling random filename /mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/Pos01_img_000000078.png
Doing intesity normalized psf placement


TypeError: unsupported operand type(s) for +=: 'Tensor' and 'tuple'

### Simulation data so far

In [9]:
data_gen = DataSimulator(psf_params, simulation_params, hardware_params)

In [10]:
random_filename = random.choice(data_gen.cell_mask_filenames)
random_cell_mask = imread(random_filename)
prob_map, cell_masks_batch = generate_probmap_cells(random_cell_mask, 128, 128, 0.003125, 10, 0.05)

In [11]:
prob_map.shape, cell_masks_batch.shape

((128, 128, 128), (128, 128, 128))

In [12]:
index = 14
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probability map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask crop')
plt.show()

In [22]:
S, X_os, Y_os, Z, I = data_gen.sampling(batch_size=128, prob_map=gpu(prob_map), local_context=None, iter_num=None,
                                        train_size=128)

In [23]:
index = 14
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(S[index][0].cpu().numpy())
ax[0].set_title('Dot location')

x_os_img = ax[1].imshow(X_os[index][0].cpu().numpy())
ax[1].set_title('X offset')
fig.colorbar(x_os_img, ax=ax[1])
y_os_img = ax[2].imshow(Y_os[index][0].cpu().numpy())
ax[2].set_title('Y offset')
fig.colorbar(y_os_img, ax=ax[2])
z_vals_img = ax[3].imshow(Z[index][0].cpu().numpy())
ax[3].set_title('Z value')
fig.colorbar(z_vals_img, ax=ax[3])
i_vals_img = ax[4].imshow(I[index][0].cpu().numpy())
ax[4].set_title('Intensity Value')
fig.colorbar(i_vals_img, ax=ax[4])
ax[5].imshow(cell_masks_batch[index])
ax[5].set_title('cell mask')
plt.show()

#### PSF placement

In [24]:
imgs_sim, psfs_sim = data_gen.simulate_psfs(S, X_os, Y_os, Z, I)

Doing intesity normalized psf placement


In [25]:
imgs_sim.shape

torch.Size([128, 1, 128, 128])

In [26]:
psfs_sim.shape

torch.Size([812, 41, 41])

In [27]:
psfs_sim.sum(-1).sum(-1)

tensor([0.8998, 0.8745, 0.9936, 0.9884, 0.9878, 0.9909, 0.9834, 0.9915, 0.9782,
        0.9808, 0.9068, 0.9889, 0.8236, 0.9665, 0.9810, 0.9342, 0.9670, 0.9831,
        0.9844, 0.8949, 0.9133, 0.9781, 0.9880, 0.9039, 0.9827, 0.9874, 0.9825,
        0.9003, 0.9804, 0.9768, 0.9341, 0.9542, 0.9125, 0.9871, 0.9235, 0.9138,
        0.9330, 0.9846, 0.9879, 0.9264, 0.9312, 0.9812, 0.9811, 0.8777, 0.9842,
        0.9464, 0.9934, 0.9658, 0.9853, 0.9820, 0.9856, 0.9209, 0.9917, 0.9027,
        0.9088, 0.8966, 0.8730, 0.9906, 0.9882, 0.9926, 0.8372, 0.8994, 0.9840,
        0.9819, 0.9774, 0.8462, 0.9810, 0.9882, 0.9930, 0.9705, 0.9592, 0.9319,
        0.9379, 0.9860, 0.9770, 0.9880, 0.8951, 0.9853, 0.9730, 0.8754, 0.9679,
        0.9905, 0.9810, 0.9932, 0.9904, 0.9718, 0.9896, 0.9734, 0.9811, 0.9833,
        0.9813, 0.9077, 0.9775, 0.9837, 0.9872, 0.9114, 0.9633, 0.9308, 0.9828,
        0.9845, 0.9931, 0.9181, 0.9787, 0.8759, 0.9239, 0.9874, 0.8971, 0.9732,
        0.9909, 0.9931, 0.9858, 0.9856, 

In [16]:
index = 14
fig, ax = plt.subplots(nrows=1, ncols=5)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probabilty map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
plt.show()

#### Noise inside and outside cells

In [17]:
import edt

In [18]:

dists = np.zeros_like(cell_masks_batch)
for i in range(len(cell_masks_batch)):
    dists[i] = edt.edt(cell_masks_batch[i])

In [19]:
dists.shape

(128, 128, 128)

In [20]:
index = 12
fig, ax = plt.subplots(nrows=1, ncols=3)
ax[0].imshow(cell_masks_batch[index])
ax[0].set_title('Cell mask')
ax[1].imshow(dists[index])
ax[1].set_title('Edt')
ax[2].imshow(prob_map[index])
ax[2].set_title('Prob map')
plt.show()

In [23]:
import json
import pathlib
from pathlib import Path

In [24]:
edt_dump_path = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/edt_noise.json')

In [25]:
with open(edt_dump_path, 'r') as fp:
    edt_map = json.load(fp)

### Edt based noise map

In [21]:
edt_map = {'mean': {
  0: 108.185,
  1: 113.56664410019513,
  2: 114.71611762104797,
  3: 115.68981552065864,
  4: 115.99452166802944,
  5: 116.43371444769915,
  6: 117.20052654673103,
  7: 118.23529411764706},
 'stddev': {
  0: 8.04400374531753,
  1: 8.039596318233265,
  2: 7.881760738448235,
  3: 7.770427535763205,
  4: 7.712307824702853,
  5: 7.704092592917589,
  6: 7.48464449317275,
  7: 8.069765006890597},
 'counts': {1: 25111, 2: 18092, 3: 13118, 4: 12230, 5: 10583, 6: 2279, 7: 17},
'fits': {1: {'alpha': 198.17018208141224, 'loc': -1.5, 'beta': 0.5806455991089693},
         2: {'alpha': 211.7177497677604, 'loc': -1.5, 'beta': 0.5489200492095204},
         3: {'alpha': 219.90042254901124, 'loc': -1.5, 'beta': 0.5329221934284345},
         4: {'alpha': 224.80044784851103, 'loc': -1.5, 'beta': 0.5226614216854536},
         5: {'alpha': 226.36710216849957, 'loc': -1.5, 'beta': 0.5209843361422435},
         6: {'alpha': 242.36910846887733, 'loc': -1.5, 'beta': 0.489751054895568},
         7: {'alpha': 209.18961551350893, 'loc': -1.5, 'beta': 0.5723768544806894}}}

In [22]:
mean_map = edt_map['mean']
stddev_map = edt_map['stddev']


dists = np.zeros_like(cell_masks_batch)
for i in range(len(cell_masks_batch)):
    dists[i] = edt.edt(cell_masks_batch[i])

In [23]:
fitted_alpha_map = {key: value['alpha'] for key, value in edt_map['fits'].items()}
fitted_beta_map = {key: value['beta'] for key, value in edt_map['fits'].items()}

In [24]:
fitted_alpha_map[0] = 168.92781237414516
fitted_beta_map[0] = 0.6581863040583447

In [27]:
fitted_alpha_map = edt_map['alphas']
print(fitted_alpha_map)

{'1': 198.26783822336986, '2': 202.19717812453737, '3': 206.53614247375737, '4': 211.50892435104902, '5': 218.42672100183074, '6': 236.5096097576979, '0': 175.97058789482784}


In [28]:
fitted_beta_map = edt_map['betas']
print(fitted_beta_map)

{'1': 0.5817275038062398, '2': 0.5755147241351868, '3': 0.5680015773432711, '4': 0.5567916977275827, '5': 0.5408661434281803, '6': 0.5033149478710346, '0': 0.6266468482700739}


In [33]:
def mean_bg_mask(dists, mean_map):
    dists_copy = np.copy(dists)
    for edt_val, mean_bg in mean_map.items():
        dists_copy[dists == int(edt_val)] = mean_bg
    return dists_copy

In [34]:
def variance_bg_mask(dists, variance_map):
    dists_copy = np.copy(dists)
    for edt_val, stddev_bg in variance_map.items():
        dists_copy[dists == int(edt_val)] = stddev_bg
    return dists_copy

In [35]:
dists = np.ceil(dists)

In [36]:
#mean_bg_cells = mean_bg_mask(dists, mean_map)
#stddev_bg_cells = variance_bg_mask(dists, stddev_map)
alpha_bg_cells = mean_bg_mask(dists, fitted_alpha_map)
beta_bg_cells = variance_bg_mask(dists, fitted_beta_map)

In [37]:
index = 10
fig, ax = plt.subplots(nrows=1, ncols=2)
#ax[0].imshow(mean_bg_cells[index])
ax[0].imshow(alpha_bg_cells[index])
ax[0].set_title('Alpha values')
#ax[0].set_title('Mean bg values')
#ax[1].imshow(stddev_bg_cells[index])
ax[1].imshow(beta_bg_cells[index])
#ax[1].set_title('Std dev bg values')
ax[1].set_title('Beta values')
plt.show()

In [38]:
#mean_t = torch.from_numpy(mean_bg_cells)
#stddev_t = torch.from_numpy(stddev_bg_cells)
alpha_t = torch.from_numpy(alpha_bg_cells)
beta_t = 1.0/torch.from_numpy(beta_bg_cells)

In [39]:
alpha_t.shape, beta_t.shape

(torch.Size([128, 128, 128]), torch.Size([128, 128, 128]))

In [40]:
#m = torch.distributions.Normal(loc=mean_t, scale=stddev_t)
m = torch.distributions.gamma.Gamma(concentration=alpha_t, rate=beta_t)

In [41]:
sample = m.sample()

In [42]:
index = 52
fig, ax = plt.subplots(nrows=1, ncols=3)
#ax[0].imshow(mean_bg_cells[index])
ax[0].imshow(alpha_bg_cells[index])
ax[0].set_title('Alpha bg cells')
#ax[0].set_title('Mean bg values')
#ax[1].imshow(stddev_bg_cells[index])
#ax[1].set_title('Std dev bg values')
ax[1].imshow(beta_bg_cells[index])
ax[1].set_title('Beta bg cells')
img = ax[2].imshow(sample[index].cpu().numpy(), cmap='gray')
ax[2].set_title('Sampled bg values')
fig.colorbar(img, ax=ax[2])
plt.show()

In [43]:
index = 15
plt.figure()
plt.imshow(sample[index].cpu().numpy(), cmap='gray')
plt.show()

#### Sample using fitted gamma distributions done. probably works better than gaussian sampling

#### Dot overlay

In [44]:
baseline = 103.0
e_per_adu = 0.39
qe = 0.95
bg_photons = (sample - baseline) * e_per_adu / qe

In [45]:
bg_photons = bg_photons[:, None]

In [46]:
bg_photons = torch.clamp(bg_photons, min=0.0)

In [47]:
imgs_sim.shape, bg_photons.shape

(torch.Size([128, 1, 128, 128]), torch.Size([128, 1, 128, 128]))

In [48]:
bg_photons = bg_photons.to('cuda:0')

In [49]:
after_dots = imgs_sim + bg_photons

In [50]:
index = 10
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probabilty map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
ax[5].imshow(after_dots[index][0].cpu().numpy(), cmap='gray')
ax[5].set_title('After overlaying dots')
plt.show()

#### Adding Camera noise model

In [51]:
RN = 2.2

In [52]:
after_dots = torch.distributions.Poisson(after_dots * qe).sample()

In [53]:
zeros = torch.zeros_like(after_dots)

In [54]:
readout_noise = torch.distributions.Normal(zeros, zeros + RN).sample()

In [55]:
after_dots = after_dots + readout_noise

In [56]:
final_dots_for_net = torch.clamp((after_dots/ e_per_adu) + baseline, min=0)

In [57]:
index = 80
fig, ax = plt.subplots(nrows=1, ncols=7)
ax[0].imshow(sample[index].cpu().numpy(), cmap='gray')
ax[0].set_title('Sampled background ADU')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
ax[5].imshow(after_dots[index][0].cpu().numpy(), cmap='gray')
ax[5].set_title('After overlaying dots (photons)')
ax[6].imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
ax[6].set_title('After camer noise (ADU)')
plt.tight_layout()
plt.show()